# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
import time
import copy
import json
## Ruberic:Package Imports :-> All the necessary packages and modules are imported in the first cell of the notebook

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
dirs=[train_dir,valid_dir,test_dir]

In [3]:
image_datasets=[]
dataloaders=[]
data_transforms=[
    transforms.Compose([
        transforms.RandomRotation(90),
        transforms.RandomResizedCrop(255),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),]),# Refrence for syntax and normalize value from here https://pytorch.org/docs/stable/torchvision/transforms.html
## Ruberic:Training data augmentation :-> torchvision transforms are used to augment the training data with random scaling, rotations, mirroring, and/or cropping
    transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),]),
    transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),])# Refrence for syntax and normalize value from 'Loading Image Data Exercise' of last chapter
            ]
## Ruberic:Data normalization:-> The training, validation, and testing data is appropriately cropped and normalized
for i in range(3):
    datasets_var=datasets.ImageFolder(dirs[i], transform=data_transforms[i])
    dataloaders.append(torch.utils.data.DataLoader(datasets_var, batch_size=64))
    image_datasets.append(datasets_var)
## Ruberic:Data batching :-> The data for each set is loaded with torchvision's DataLoader
## Ruberic:Data loading :-> The data for each set (train, validation, test) is loaded with torchvision's ImageFolder

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [5]:
model = models.vgg16(pretrained=True)
## Ruberic:Pretrained Network :-> A pretrained network such as VGG16 is loaded from torchvision.models and the parameters are frozen

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:05<00:00, 107472644.35it/s]


In [6]:
hidden_units_count=600
model.classifier = nn.Sequential( nn.Linear(25088, hidden_units_count), nn.ReLU(), nn.Linear(hidden_units_count, 102)) #I got reference of syntax from here:-> https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html
## Ruberic:Feedforward Classifier :-> A new feedforward network is defined for use as a classifier using the features as input

In [7]:
criterion = nn.CrossEntropyLoss() # it is addition of LogSotfmax function and NLLloss funcion
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
def my_nn(model, trainloader, testloader, epochs, criterion, optimizer, device):
    device = 'cuda' if device == 'gpu' else 'cpu'
    epochs = 30
    steps = 0

    train_losses, test_losses = [], []
    for e in range(epochs):
        running_loss = 0
        for images, labels in trainloader:

            optimizer.zero_grad()
            log_ps = model(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        else:
            test_loss = 0
            accuracy = 0
            with torch.no_grad():
                model.eval()
                for images, labels in testloader:
                    log_ps = model(images)
                    test_loss += criterion(log_ps, labels)

                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
        model.train()
## Ruberic:Validation Loss and Accuracy :-> During training, the validation loss and accuracy are displayed        
my_nn(model, dataloaders[0], dataloaders[1], 3, criterion, optimizer, 'gpu') 
# This Function Is Refered from 'Inference and Validation Exersice' of pytorch lesson from neural network chapter 
## Ruberic:Training the network :-> The parameters of the feedforward classifier are appropriately trained, while the parameters of the feature network are left static

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
def check_accuracy_of_my_nn(testloader, device):    
    test_loss = 0
    accuracy = 0
    device = 'cuda' if device == 'gpu' else 'cpu'
    with torch.no_grad():
        for images, labels in testloader:
            log_ps = model(images)
            test_loss += criterion(log_ps, labels)
            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))
    print('Accuracy: %d %%' % (100 * accuracy))
check_accuracy_of_my_nn(dataloaders[0], 'gpu')# This Function Is Refered from 'Inference and Validation Exersice' of pytorch lesson from neural network chapter 
## Ruberic:Testing Accuracy :-> The network's accuracy is measured on the test data        

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
model.class_to_idx = image_datasets[0].class_to_idx
#image_datas et to model 

In [ ]:
torch.save({'arch':'vgg16','state_dict':model.state_dict(),'class_to_idx':model.class_to_idx},'checkpoint.pth')
## Ruberic:Saving the model :-> The trained model is saved as a checkpoint along with associated hyperparameters and the class_to_idx dictionary        


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
checkpoint = torch.load('checkpoint.pth')
checkpoint.keys()
model = models.vgg16(pretrained=True)
classifier = nn.Sequential( nn.Linear(25088, hidden_units_count), nn.ReLU(), nn.Linear(hidden_units_count, 102)) #I got reference of syntax from here:-> https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html
## Ruberic:Loading checkpoints :-> There is a function that successfully loads a checkpoint and rebuilds the model
model.classifier = classifier
model.class_to_idx = checkpoint['class_to_idx']
model.load_state_dict(checkpoint['state_dict'])

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(path_to_image):
    image = Image.open(path_to_image)
    #Refrence is from https://pillow.readthedocs.io/en/latest/reference/Image.html
    image=image.resize((image.width*256//image.height,256)) if image.width > image.height else image.resize((256,image.height*256//image.width))    
    #in above expression,I have resize the images where the shortest side is 256 pixels, keeping the aspect ratio.
    return ((((np.array(image.crop(((image.width-224)/2,(image.height-224)/2,(image.width-224)/2+224, (image.height-224)/2+224))))/255) # image converted to np array from PIL
             - np.array([0.485, 0.456, 0.406]))#substracting mean from resulting np array
            /np.array([0.229, 0.224, 0.225])#Dividing by Standard Deviation
           ).transpose((2, 0, 1)) #transposing it to make third channel (color channel) to first and remaining both maintaining the sequence
process_image(train_dir+'/18/image_04266.jpg')
## Ruberic:Image Processing :-> The process_image function successfully converts a PIL image into an object that can be used as input to a trained model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    image = np.clip(image, 0, 1)
    ax.imshow(image)
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path):
    return (torch.exp(model.forward(
        (torch.from_numpy( process_image(image_path)).type(torch.FloatTensor)).unsqueeze_(0) #Converting np array to float tensor using from_numpy 
        ))).topk(5) #considering top 5 result refer from here https://pytorch.org/docs/master/torch.html#torch.topk
predict(train_dir+'/18/image_04266.jpg')
## Ruberic:Class Prediction :-> The predict function successfully takes the path to an image and a checkpoint, then returns the top K most probably classes for that image

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
def display_img(image_path):
    category = [] #Empty list
    name_list=[] #Empty list
    flower=[] #Empty list
    imshow(process_image(image_path), plt.subplot(2,1,1)) #Displaying Image
    probs, classes = predict(image_path) #predicting image with top 5 results
    for i in classes[0]:
        flower.append(i) #adding new element to flower
    for x in flower:
        for category_name, category_value in checkpoint['class_to_idx'].items(): #We are getting category key and value from class_to_idx dictonary
            category.append(category_name) if category_value == x else continue  #Storing Category Names if present in flower list
    for i in category:
        name_list.append(cat_to_name[i]) #if category value is present in cat_to_name then storing the label name to store it into name_list so that it would be use to print on bar chart
## Ruberic:Sanity Checking with matplotlib :-> A matplotlib figure is created displaying an image and its associated top 5 most probable classes with actual flower names
    ax = plt.subplot(2,1,2)
    ax.barh(name_list, probs[0]) #printing bar chart with possible naame_list and value
    plt.show()
display_img(train_dir+'/18/image_04266.jpg')